In [5]:
import pandas as pd
import numpy as np

In [6]:
d_gcs=pd.read_csv("gcs_scores_updated.csv")
d_gcs=d_gcs.drop(columns=['nursingchartcelltypecat','nursingchartcelltypevallabel'])
d_gcs=d_gcs.drop_duplicates()
d_gcs=d_gcs.rename(columns={'nursingchartentryoffset':'offset'})

In [7]:
labs=pd.read_csv("labs_morevars.csv")
labs=labs.drop_duplicates()
labs=labs.rename(columns={'labresultoffset':'offset'})

In [8]:
cardiovas=pd.read_csv("drugate_norm_updated.csv")
cardiovas.drop_duplicates()
cardiovas=cardiovas.drop(columns=['nursingchartvalue','SOFA_cardio'])

In [9]:
#First merge
labs_cardio=pd.merge(labs,cardiovas,how="outer",on=['patientunitstayid','offset']).drop_duplicates()
labs_cardio.to_csv("labs_cardio_interim.csv",index=False)

In [10]:
del labs
del cardiovas

In [11]:
training_build=pd.merge(labs_cardio,d_gcs,how="outer",on=['patientunitstayid','offset']).drop_duplicates()
del d_gcs

In [13]:
#Initial length due to replication
len(training_build)

12771645

In [14]:
#To correct the replication of offsets for same patients
training_build=training_build.groupby(['patientunitstayid','offset'],as_index=False).max().drop_duplicates()

In [15]:
len(training_build)

5902370

In [16]:
training_build=training_build.groupby(['patientunitstayid'],as_index=False).apply(pd.DataFrame.sort_values,'offset').reset_index()
training_build=training_build.drop(columns=['level_0','level_1'])
training_build=training_build.drop(columns=['Norepinephrine','Epinephrine','Dopamine','Dobutamine'])
training_build_filtered=training_build.dropna(subset=['paO2_FiO2','platelets_x_1000','total_bilirubin','urinary_creatinine','creatinine','HCO3','pH','paCO2','direct_bilirubin','excess','ast','bun','calcium','GCS_Score'],how='all')
training_build_filtered['label']=np.nan

In [24]:
sepsis_labels=pd.read_csv("24_hour_sepsis.csv")
sepsis_labels['tsepsis']=sepsis_labels[['tsus','tsofa']].min(axis=1)

,patientunitstayid,tsofa,tsus,flag,tsepsis
0,242040,69.0,188.0,1,69.0
1,242154,270.0,8589.0,0,270.0
2,242290,165.0,165.0,1,165.0
3,242434,1340.0,1.0,0,1.0
4,242544,9.0,31.0,1,9.0


In [46]:
final_build=pd.merge(training_build_filtered,sepsis_labels,how='left',left_on=['patientunitstayid','offset'],right_on=['patientunitstayid','tsepsis'])
final_build.label=final_build.flag
final_build=final_build.drop(columns=['tsofa','tsus','tsepsis','flag'])
#After the initial sepsis=1 flag, all the labels for that patient is given label=1, all before that is 0
final_build['label']=final_build.groupby(['patientunitstayid'])['label'].ffill()
final_build['label']=final_build['label'].fillna(0)

In [51]:
final_build.to_csv("training_table.csv",index=False)

In [54]:
final_build_nonneg=final_build.loc[~(final_build['offset']<0)]
final_build_nonneg.to_csv("training_table_nonneg.csv",index=False)